In [2]:
# --- Step 1. Setup ---
# Install required packages
!pip install folium geopy tqdm

import pandas as pd
import folium
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from tqdm import tqdm
import time

# --- Step 2. Load the dataset ---
# Upload your CSV to Colab first (Files tab → Upload) or mount Google Drive
# Example if you uploaded manually:
df = pd.read_csv("/content/Parking_Tickets (3).csv")

print("Number of rows:", len(df))
df.head()

# --- Step 3. Clean and combine address fields ---
# Create a full address column for geocoding
df["FullAddress"] = df["Location"].astype(str) + ", Charlottesville, VA"
df = df.dropna(subset=["FullAddress"])

print("Sample addresses:")
print(df["FullAddress"].head())

# --- Step 4. Geocode a sample (to stay within API limits) ---
# Nominatim allows ~1 request per second, so we’ll just do a small random sample.
sample_df = df.sample(200)  # adjust sample size as desired

geolocator = Nominatim(user_agent="cville_tickets")

def geocode_address(address):
    """Convert address → (latitude, longitude) using OpenStreetMap Nominatim."""
    try:
        location = geolocator.geocode(address)
        time.sleep(1)  # rate-limit safe
        if location:
            return pd.Series([location.latitude, location.longitude])
    except Exception as e:
        return pd.Series([None, None])
    return pd.Series([None, None])

tqdm.pandas()
sample_df[["Latitude", "Longitude"]] = sample_df["FullAddress"].progress_apply(geocode_address)

# Drop rows that failed geocoding
sample_df = sample_df.dropna(subset=["Latitude", "Longitude"])
print("Geocoded rows:", len(sample_df))

# --- Step 5. Create the heatmap ---
# Center map roughly on Charlottesville
m = folium.Map(location=[38.0293, -78.4767], zoom_start=13)

heat_data = sample_df[["Latitude", "Longitude"]].values.tolist()
HeatMap(heat_data, radius=10, blur=12, min_opacity=0.4).add_to(m)

# --- Step 6. Display and save ---
m.save("charlottesville_tickets_heatmap.html")
m


/tmp/ipython-input-1925443201.py:15: DtypeWarning: Columns (1,4,5,7,8,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/Parking_Tickets (3).csv")


Number of rows: 506311
Sample addresses:
0             100 W WATER ST, Charlottesville, VA
1              22 14TH ST NW, Charlottesville, VA
2              100 5TH ST SW, Charlottesville, VA
3    1700 JEFFERSON PARK AVE, Charlottesville, VA
4               100 1ST ST S, Charlottesville, VA
Name: FullAddress, dtype: object


100%|██████████| 200/200 [06:38<00:00,  1.99s/it]

Geocoded rows: 154
